<h3> Imports </h3>

In [34]:
import ee
import os
import numpy as np
import geemap
ee.Initialize()

In [21]:
IPIS = ee.FeatureCollection("users/rishiAgarwal/Congo_Active_Mines")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
visualization = {
    "min": 0,
    "max": 0.3,
    "bands": ["B4","B3","B2"]
}
visParams = {'color': 'blue'}
# cloud mask sentinel
def maskS2clouds(image):
    qa = image.select('QA60')

    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000).select("B.*") \
        .copyProperties(image, ["system:time_start"])

<h3> Essential Functions </h3>

In [28]:
def convertToFeatureCollection(csv):
    points = np.genfromtxt(csv, delimiter=',', skip_header=1)
    feature_array = points
    region_list = []
    for row in feature_array:
        region = ee.Geometry.Polygon([[[float(row[0]), float(row[3])],
                                      [float(row[0]), float(row[1])],
                                      [float(row[2]), float(row[1])],
                                      [float(row[2]), float(row[3])]]])
        f = ee.Feature(region).set('elevation score', float(row[17])).set('band score', float(row[18]))
        region_list.append(f)

    # Wrap feature list in a Feature Collection
    fc = ee.FeatureCollection(region_list)
    return fc

In [45]:
def visArea(min_lon, max_lon, max_lat, min_lat):
    Map = geemap.Map()
    area = ee.Geometry.Rectangle(min_lon, min_lat, max_lon, max_lat)
    filtered = s2 \
    .filter(ee.Filter.bounds(area)) \
    .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
    .map(maskS2clouds) \
    .select('B.*')
    composite = filtered.median().clip(area)
    return composite

In [20]:
def nearestNeighbor(result_filepath, ipis_dataset):
    os.system('python3 nearest_neighbors.py ' + result_filepath + ' ' + ipis_dataset)

<h3> CSVs Filepaths </h3>

In [22]:
r_26_n7 = "bigRegion_csvs/26_-7_passing.csv"
r_27_n7 = "bigRegion_csvs/27_-7_passing.csv"
r_28_n7 = "bigRegion_csvs/28_-7_passing.csv"
r_29_n7 = "bigRegion_csvs/29_-7_passing.csv"

<h3> Ipis Dataset Filepaths </h3>

In [23]:
ipis_26_n7 = "Ipis_datasets/ipis_26_-7.csv"
ipis_27_n7 = "Ipis_datasets/ipis_27_-7.csv"
ipis_28_n7 = "Ipis_datasets/ipis_28_-7.csv"
ipis_29_n7 = "Ipis_datasets/ipis_29_-7.csv"

<h3> Visualizations </h3>

In [58]:
table = convertToFeatureCollection(r_26_n7)
composite = visArea(26, 27, -7, -8)
Map = geemap.Map()
Map.addLayer(composite, visualization, '26_-7 Composite Image')
Map.addLayer(IPIS, visParams, "mines", opacity=0.5)
Map.addLayer(table, {'color': 'red'}, "26_-7_passing regions", opacity=0.5)
Map.centerObject(composite, 9)
Map


Map(center=[-7.500090808548613, 26.499999999999957], controls=(WidgetControl(options=['position', 'transparent…

<h3> Nearest Neighbor Information </h3>

In [56]:
nearestNeighbor(r_27_n7, ipis_27_n7)

% within 1km of a mapbox point: 67.95580110497238
% within 5km of a mapbox point: 90.60773480662984
% within 10km of a mapbox point: 97.79005524861878
